In [18]:
import numpy as np
import pandas as pd
import random
from sklearn.utils import shuffle
# from sklearn.model_selection import train_test_split
from sklearn.model_selection import RepeatedKFold
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE
from sklearn.base import clone

In [19]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

In [20]:
from keras.models import load_model
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Flatten
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, roc_auc_score, \
        f1_score, precision_score, recall_score
import matplotlib.pyplot as plt

# Load Data

In [21]:
df = pd.read_csv("gaze_labels.csv", index_col = "Participant_ID")

In [22]:
df.head()

,PHQ_Score,PHQ_Binary
Participant_ID,,
300,2,0
301,3,0
302,4,0
303,0,0
304,6,0


In [23]:
X = np.load("avg_gaze.npy")
y = df['PHQ_Binary']

# Prepare data

In [24]:
# split data into the 4 vectors
X_f0 = X[:,0:3]
X_f1 = X[:,3:6]
X_fh0 = X[:,6:9]
X_fh1 = X[:,9:12]

In [25]:
def train_test(X, y, testfile='test_split_Depression_AVEC2017.csv'):
    test_participants = pd.read_csv(testfile)['participant_ID'].values
    X_train = []
    X_test = []
    y_train = []
    y_test = []
    for i in range(y.shape[0]):
        participant_no = y.index[i]
        if participant_no in test_participants:
            X_test.append(X[i])
            y_test.append(y[participant_no])
        else:
            X_train.append(X[i])
            y_train.append(y[participant_no])
    return np.array(X_train), np.array(X_test), np.array(y_train), np.array(y_test)

In [26]:
# edit the X parameter to obtain the train test sets for that feature vector
X_train, X_test, y_train, y_test = train_test(X_fh1, y, '../test_split_Depression_AVEC2017.csv')

In [27]:
X_test.shape

(45, 3)

In [28]:
RANDOM_STATE = 42

In [29]:
def undersampling(X_train, y_train):
    random.seed(RANDOM_STATE)
    
    neg_list = [i for i in range(len(y_train)) if y_train[i] == 0]
    pos_list = [i for i in range(len(y_train)) if y_train[i] == 1]
    
    if len(neg_list) < len(pos_list):
        minority_list = neg_list
        majority_list = pos_list
    else:
        minority_list = pos_list
        majority_list = neg_list
        
    sampled_list = random.sample(majority_list, len(minority_list))
    
    final_list = sampled_list + minority_list
    
    X_train_us = []
    y_train_us = []
    
    for i in final_list:
        X_train_us.append(X_train[i])
        y_train_us.append(y_train[i])
                                 
    return np.array(X_train_us), np.array(y_train_us)

X_train, y_train = undersampling(X_train, y_train)
X_train, y_train = shuffle(X_train, y_train, random_state=RANDOM_STATE)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((82, 3), (45, 3), (82,), (45,))

In [30]:
# np.save("X_train_g01.npy", X_train)
# np.save("X_test_g01.npy", X_test)
# np.save("y_train_g01.npy", y_train)

# Testing different models using k-cross

# k cross

In [31]:
def k_cross(input_model, X=X_train, y=y_train, k=4, n=3, random_state=42):
    f1_scores = []
    recall_scores = []
    rkf = RepeatedKFold(n_splits=k, n_repeats=n, random_state=random_state)
    for train_index, val_index in rkf.split(X):
        X_train, X_val = X[train_index], X[val_index]
        y_train, y_val = y[train_index], y[val_index]
        
        model = clone(input_model)
        model.fit(X_train, y_train) 
        
        y_pred = model.predict(X_val)
        f1 = f1_score(y_val, y_pred)
        f1_scores.append(f1)
        recall = recall_score(y_val, y_pred)
        recall_scores.append(recall)
        
    return f1_scores, recall_scores

# Logistic Regression

In [32]:
def find_best_logreg_model(power):
    best_f1_model = None
    best_recall_model = None
    best_f1 = -1
    best_recall = -1
    
    for i in range(power + 1):
        model = LogisticRegression(n_jobs=3, C=10**i)
        
        f1_scores, recall_scores = k_cross(model)
        f1 = np.mean(f1_scores)
        recall = np.mean(recall_scores)

        if f1 > best_f1:
            print(f"power: {i}: f1 = {f1} > best f1 = {best_f1}")
            best_f1 = f1
            best_f1_model = model

        if recall > best_recall:
            print(f"power: {i}: recall = {recall} > best recall = {best_recall}")
            best_recall = recall
            best_recall_model = model
            
    print("F1 score:", best_f1)
    
    return best_f1_model, best_recall_model

best_logreg_f1_model, best_logreg_recall_model = find_best_logreg_model(6)

power: 0: f1 = 0.26766146688560477 > best f1 = -1
power: 0: recall = 0.41574074074074074 > best recall = -1
power: 1: f1 = 0.3748645068491396 > best f1 = 0.26766146688560477
power: 1: recall = 0.4383466070966071 > best recall = 0.41574074074074074
power: 2: f1 = 0.3844103533028807 > best f1 = 0.3748645068491396
power: 2: recall = 0.4449673012173012 > best recall = 0.4383466070966071
F1 score: 0.3844103533028807


# Decision Tree

In [33]:
def find_best_tree_model(upper_depth, upper_leaf):
    best_f1_model = None
    best_recall_model = None
    best_f1 = -1
    best_recall = -1
    
    for depth in range(1, upper_depth + 1):
        for leaf in range(1, upper_leaf + 1):
            model = DecisionTreeClassifier(random_state=RANDOM_STATE, max_depth=depth, min_samples_leaf=leaf) 
            
            f1_scores, recall_scores = k_cross(model)
            f1 = np.mean(f1_scores)
            recall = np.mean(recall_scores)
            
            if f1 > best_f1:
                print(f"depth: {depth}, leaf: {leaf}: f1 = {f1} > best f1 = {best_f1}")
                best_f1 = f1
                best_f1_model = model
                
                print("F1: ",f1)
                print("Recall: ", recall)
                
            if recall > best_recall:
                print(f"depth: {depth}, leaf: {leaf}: recall = {recall} > best recall = {best_recall}")
                best_recall = recall
                best_recall_model = model
                
            
    print("F1 score:", best_f1)
    
    return best_f1_model, best_recall_model

best_tree_f1_model, best_tree_recall_model = find_best_tree_model(20, 30)

depth: 1, leaf: 1: f1 = 0.32020702131866313 > best f1 = -1
F1:  0.32020702131866313
Recall:  0.3600281662781663
depth: 1, leaf: 1: recall = 0.3600281662781663 > best recall = -1
depth: 1, leaf: 5: f1 = 0.45400723265859 > best f1 = 0.32020702131866313
F1:  0.45400723265859
Recall:  0.500853082103082
depth: 1, leaf: 5: recall = 0.500853082103082 > best recall = 0.3600281662781663
depth: 1, leaf: 8: f1 = 0.48178501043636784 > best f1 = 0.45400723265859
F1:  0.48178501043636784
Recall:  0.5216864154364154
depth: 1, leaf: 8: recall = 0.5216864154364154 > best recall = 0.500853082103082
depth: 1, leaf: 10: f1 = 0.48368434566903645 > best f1 = 0.48178501043636784
F1:  0.48368434566903645
Recall:  0.5216864154364154
depth: 2, leaf: 1: recall = 0.5556397306397306 > best recall = 0.5216864154364154
depth: 2, leaf: 4: f1 = 0.5168373980998919 > best f1 = 0.48368434566903645
F1:  0.5168373980998919
Recall:  0.61508029008029
depth: 2, leaf: 4: recall = 0.61508029008029 > best recall = 0.555639730639

# RF

In [34]:
def find_best_forest_model(n_estimators):
    best_f1_model = None
    best_recall_model = None
    best_f1 = -1
    best_recall = -1
    
    for estimator in range(1, n_estimators + 1):
        model = RandomForestClassifier(random_state=RANDOM_STATE, n_estimators=estimator) 
        
        f1_scores, recall_scores = k_cross(model)
        f1 = np.mean(f1_scores)
        recall = np.mean(recall_scores)

        if f1 > best_f1:
            print(f"estimator: {estimator}: f1 = {f1} > best f1 = {best_f1}")
            best_f1 = f1
            best_f1_model = model

        if recall > best_recall:
            print(f"estimator: {estimator}: recall = {recall} > best recall = {best_recall}")
            best_recall = recall
            best_recall_model = model
    print("F1 score:", best_f1)
    
    return best_f1_model, best_recall_model

best_forest_f1_model, best_forest_recall_model = find_best_forest_model(30)

estimator: 1: f1 = 0.4114691809738249 > best f1 = -1
estimator: 1: recall = 0.4188616938616938 > best recall = -1
estimator: 3: f1 = 0.46462463056553777 > best f1 = 0.4114691809738249
estimator: 3: recall = 0.47227078477078477 > best recall = 0.4188616938616938
estimator: 5: f1 = 0.5055621601667503 > best f1 = 0.46462463056553777
estimator: 5: recall = 0.5202845765345765 > best recall = 0.47227078477078477
estimator: 7: f1 = 0.5175376440015018 > best f1 = 0.5055621601667503
estimator: 7: recall = 0.5290970603470603 > best recall = 0.5202845765345765
estimator: 9: f1 = 0.5233194345625359 > best f1 = 0.5175376440015018
estimator: 9: recall = 0.539408508158508 > best recall = 0.5290970603470603
F1 score: 0.5233194345625359


# SVM with Grid Search

In [37]:
tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4], 'C': [1, 10, 100]},
                    {'kernel': ['poly'], 'degree': [3, 4, 5, 6, 7], 'C': [1, 10, 100]},
                    {'kernel': ['linear'], 'C': [1, 10, 100, 1000, 10000, 100000]}]

svm_model_cv = GridSearchCV(SVC(), tuned_parameters, cv=10, scoring='f1', verbose=1, n_jobs=4)

f1_scores, recall_scores = k_cross(svm_model_cv)
print(f"f1 mean score: {np.mean(f1_scores)}")
print(f"recall mean score: {np.mean(recall_scores)}")
best_svm_model = svm_model_cv.fit(X_train, y_train)
print(f"Best SVM parameters: {best_svm_model.best_params_}")

Fitting 10 folds for each of 27 candidates, totalling 270 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done 235 tasks      | elapsed:    0.4s
[Parallel(n_jobs=4)]: Done 270 out of 270 | elapsed:    0.6s finished


Fitting 10 folds for each of 27 candidates, totalling 270 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done 235 tasks      | elapsed:    0.4s
[Parallel(n_jobs=4)]: Done 270 out of 270 | elapsed:    0.7s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.


Fitting 10 folds for each of 27 candidates, totalling 270 fits


...

[Parallel(n_jobs=4)]: Done 235 tasks      | elapsed:    0.4s
[Parallel(n_jobs=4)]: Done 270 out of 270 | elapsed:    0.6s finished


f1 mean score: 0.4746033044910453
recall mean score: 0.6467560217560218
Fitting 10 folds for each of 27 candidates, totalling 270 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done 235 tasks      | elapsed:    0.4s


Best SVM parameters: {'C': 100, 'degree': 7, 'kernel': 'poly'}


[Parallel(n_jobs=4)]: Done 270 out of 270 | elapsed:    0.7s finished
